# Neural network Tuning

In [21]:
# Importing necesary packages
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV 
from keras.models import Sequential
from keras.layers import Dense
from plotly import express as px
import warnings
warnings.filterwarnings('ignore')

from keras.wrappers.scikit_learn import KerasClassifier

# part 1 - Tuning batch and epoch

In [22]:
# load the dataset for India diabetes
data = pd.read_csv('diabetes.csv')

In [23]:
#viewing data
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [24]:
# split into predictors and response
X = data.iloc[:,0:8] #predictors
Y = data.iloc[:,8] #response

In [25]:
# Function to create model,for KerasClassifier
def create_my_model():
    #defining my model
    mymodel = Sequential()
    mymodel.add(Dense(12, input_dim=8, activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))
    
    # Compile the model
    mymodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return mymodel

In [26]:
# create model
model = KerasClassifier(build_fn=create_my_model)

In [27]:
# define the grid search parameters
batchSize = [10, 20, 40, 60, 80, 100] # يجرب اكثر من حجم عينة ويستعملها في القريد سيرش
epochs = [10, 30, 50] # كم عدد المرات اللي راح يتدرب فيها المودل

In [28]:
parameter_grid = dict(batch_size=batchSize, epochs=epochs) # حدننا الباراميترز كيف تكون


mygrid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = mygrid.fit(X, Y)

Epoch 1/50
39/39 [==============================] - 1s 1ms/step - loss: 7.0612 - accuracy: 0.4505
Epoch 2/50
39/39 [==============================] - 0s 2ms/step - loss: 4.8837 - accuracy: 0.4870
Epoch 3/50
39/39 [==============================] - 0s 1ms/step - loss: 3.4108 - accuracy: 0.5026
Epoch 4/50
39/39 [==============================] - 0s 2ms/step - loss: 2.4386 - accuracy: 0.5391
Epoch 5/50
39/39 [==============================] - 0s 2ms/step - loss: 1.7672 - accuracy: 0.5716
Epoch 6/50
39/39 [==============================] - 0s 2ms/step - loss: 1.2736 - accuracy: 0.6120
Epoch 7/50
39/39 [==============================] - 0s 1ms/step - loss: 1.0625 - accuracy: 0.6380
Epoch 8/50
39/39 [==============================] - 0s 1ms/step - loss: 0.9713 - accuracy: 0.6536
Epoch 9/50
39/39 [==============================] - 0s 2ms/step - loss: 0.9390 - accuracy: 0.6458
Epoch 10/50
39/39 [==============================] - 0s 2ms/step - loss: 0.8933 - accuracy: 0.6693
Epoch 11/50
39/39 [

In [29]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.677083 using {'batch_size': 20, 'epochs': 50}


# part 2- Optimizing Activation Function

In [30]:
# Function to create model
def create_my_model(optimizer='adam'):
    # create model
    mymodel = Sequential()
    mymodel.add(Dense(12, input_dim=8, activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))
    # Compile model
    mymodel.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return mymodel

In [31]:
# create model
model = KerasClassifier(build_fn=create_my_model, epochs=50, batch_size=10)

In [32]:
# define the grid search parameters
optimizer = ['SGD','Adadelta', 'RMSprop', 'Adagrad','Adam'] #Nadam
parameter_grid = dict(optimizer=optimizer)

In [33]:
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

Epoch 1/50
77/77 [==============================] - 1s 2ms/step - loss: 8.2247 - accuracy: 0.5156
Epoch 2/50
77/77 [==============================] - 0s 1ms/step - loss: 2.4644 - accuracy: 0.5977
Epoch 3/50
77/77 [==============================] - 0s 2ms/step - loss: 1.8737 - accuracy: 0.5872
Epoch 4/50
77/77 [==============================] - 0s 2ms/step - loss: 1.6194 - accuracy: 0.5820
Epoch 5/50
77/77 [==============================] - 0s 2ms/step - loss: 1.4063 - accuracy: 0.5898
Epoch 6/50
77/77 [==============================] - 0s 2ms/step - loss: 1.2039 - accuracy: 0.6133
Epoch 7/50
77/77 [==============================] - 0s 1ms/step - loss: 1.0858 - accuracy: 0.6172
Epoch 8/50
77/77 [==============================] - 0s 2ms/step - loss: 1.0021 - accuracy: 0.6302
Epoch 9/50
77/77 [==============================] - 0s 1ms/step - loss: 0.9408 - accuracy: 0.6380
Epoch 10/50
77/77 [==============================] - 0s 1ms/step - loss: 0.9134 - accuracy: 0.5964
Epoch 11/50
77/77 [

In [34]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) #0.727865

Best: 0.677083 using {'optimizer': 'RMSprop'}


In [35]:
data.shape

(768, 9)

# How can you improve accuracy of the model? 

In [36]:
# i've tried to increase the epochs to 300 and this scored 0.727865 with the optimizer Adam
data = data[data['BMI'] != 0]
data = data[data['Glucose'] != 0]
data = data[data['BloodPressure'] != 0]
data.shape

(724, 9)

In [37]:
model = KerasClassifier(build_fn=create_my_model, epochs=50, batch_size=10)

In [38]:
# define the grid search parameters
optimizer = ['SGD','Adadelta', 'Nadam', 'Adadelta','Adam'] #Nadam
parameter_grid = dict(optimizer=optimizer)

In [39]:
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

Epoch 1/50
77/77 [==============================] - 1s 1ms/step - loss: 25.7824 - accuracy: 0.6471
Epoch 2/50
77/77 [==============================] - 0s 1ms/step - loss: 7.9311 - accuracy: 0.6224
Epoch 3/50
77/77 [==============================] - 0s 1ms/step - loss: 2.9900 - accuracy: 0.5885
Epoch 4/50
77/77 [==============================] - 0s 1ms/step - loss: 1.4266 - accuracy: 0.5742
Epoch 5/50
77/77 [==============================] - 0s 1ms/step - loss: 1.2402 - accuracy: 0.5977
Epoch 6/50
77/77 [==============================] - 0s 2ms/step - loss: 1.1182 - accuracy: 0.6224
Epoch 7/50
77/77 [==============================] - 0s 1ms/step - loss: 1.0424 - accuracy: 0.6341
Epoch 8/50
77/77 [==============================] - 0s 2ms/step - loss: 0.9886 - accuracy: 0.6237
Epoch 9/50
77/77 [==============================] - 0s 2ms/step - loss: 0.9612 - accuracy: 0.6224
Epoch 10/50
77/77 [==============================] - 0s 2ms/step - loss: 0.9057 - accuracy: 0.6302
Epoch 11/50
77/77 

In [40]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) 

Best: 0.691406 using {'optimizer': 'Adam'}
